In [1]:
import scala.io.Source
import scala.util.Random


class KMeans(dataFile: String, k: Int, maxIterations: Int) {
    
    private val data: Array[Array[Double]] = readData(dataFile)
    // Initialiser du tableau à deux dimensions data à l'aide des données 
    // du fichier data.iris
    private val nbColumns: Int = 4
    // Initialisation de la variable nbColumns qui va prendre la valeur 4
    private val nbLines = data.length
    // Initialisation de la variable NbLines qui va prendre la valeur 150
    private var centroids: Array[Array[Double]] = initCentroids(k)
    // Initialiser de manière aléatoire k centroïdes distincts
    
    def readData(filename: String): Array[Array[Double]] = {
        // Cette méthode lit les données à partir du nom de fichier 
        // intégré dans la variable filename et stocke les valeurs 
        // de type double dans la matrice data
        val lines = scala.io.Source.fromFile(filename).getLines.toArray
        // Création de lines, un tableau de string intégrant les lignes 
        // du fichier filename
        val data = Array.ofDim[Double](lines.length, 4)
        // Création de data, un tableau de double de dimension (lines.lenght, 4)
        
        for (i <- 0 until lines.length) {
            // Pour i lignes de lines
            val fields = lines(i).split(",")
            // Tokeniser les éléments de tableau lines dans fields
            for (j <- 0 until 4) {
                // Pour i allant de 0 à 3 
                data(i)(j) = fields(j).toDouble
                // Enregistrer fields(j) dans data(i)(j)
            }
        }
        data
        // Retourner data
    }
    
    private def initCentroids(k: Int): Array[Array[Double]] = {
        // Cette méthode initialise aléatoirement k centroïdes
        val centroids = new Array[Array[Double]](k)
        // Créer un tableau de double à deux dimension de dimension (1, k)
        for (i <- 0 until k) {
            // Pour tout i de k
            centroids(i) = data(Random.nextInt(data.length))
            // Enregister i centroïdes dans le tableau centroids
        }
        centroids
        // Retourner centroids
    }
    
    private def euclideanDistance(arr1: Array[Double], arr2: Array[Double]): Double = {
        // Cette méthode calcule la distance euclidienne 
        // entre les deux tableaux arr1 et arr2
        var sum = 0.0
        for (i <- 0 until arr1.length) {
            // Pour tout i de arr1.length
            sum += math.pow(arr1(i) - arr2(i), 2)
            // Somme des i arr1 - les i arr2 au carré
        }
        math.sqrt(sum)
        // Retourner la racine carré de la somme totale
    }
    
    private def assignToClusters(): Array[Int] = {
        // Cette méthode attribue chaque point de données à un centroïde 
        // le plus proche à l'aide de la distance euclidienne pour mesurer 
        // la similarité entre chaque point de données et chaque centroïde
        val clusterAssignments = new Array[Int](data.length)
        for (i <- 0 until data.length) {
            // Pour chaque i de data :
            var minDistance = Double.MaxValue
            // Initialiser minDistance à la valeur maximale
            var closestCenterIndex = -1
            // Initialiser de closestCenterIndex à -1
            for (j <- 0 until centroids.length) {
                // Pour tout j de centroids
                val distance = euclideanDistance(data(i), centroids(j))
                // Calculer la distance euclidienne entre un point d'indice i dans data
                // et un centroïde d'indice j dans centroids
                if (distance < minDistance) {
                    // Si la distance euclidienne est inférieure 
                    // à la distance minimale enregistrée
                    minDistance = distance
                    // alors la distance est enregistrée dans mindistance
                    closestCenterIndex = j
                    // et l'indice j est enregistré dans closestCenterIndex
                }
            }
            clusterAssignments(i) = closestCenterIndex
            // Le tableau clusterAssignments permet d'enregistrer 
            // les indices des centroïdes les plus proches pour chaque i
        }
        clusterAssignments
        // Retourner clusterAssignements
    }
    
    // Calcul des nouveaux centres de chaque cluster
    private def calculateCenters(clusterAssignments: Array[Int]): Array[Array[Double]] = {
        // Cette méthode calcule les nouveaux centres grâce 
        // à la moyenne des données assignées à chaque cluster
        val newCenters = Array.ofDim[Double](k, data(0).length)
        // Création de newCenters, un tableau de double de dimension (k, 4)
        val counts = new Array[Int](k)
        // Création de counts, un tableau d'entier de dimension k
        
        for (i <- data.indices) {
            // Pour tout i de data.length :
            val clusterId = clusterAssignments(i)
            // Récupérer la valeur du cluster enregistrée dans les i clusterAssignments
            for (j <- data(i).indices) {
                // Pour tout j de data(i).length :
                newCenters(clusterId)(j) += data(i)(j)
                // Calculer la somme des i,j data et enregistrer le résultat 
                // dans les clustersId, j de newCenters
                
            }
            counts(clusterId) += 1
            // Enregistrer le nombre de points de chaque cluster
        }
        for (i <- centroids.indices) {
            // Pour tout i de centroids.indices :
            for (j <- centroids(i).indices) {
                // Pour tout j de centroids(i).indices :
                newCenters(i)(j) /= counts(i)
                // Diviser les i, j newCenters par les i counts 
                // et enregistrer le résultat dans les i, j newCenters
            }
        }
        newCenters
        //Retourner newCenters
    }
    
    def run(): Array[Int] = {
        // Cette méthode exécute l'algorithme k-means 
        var clusterIds = assignToClusters()
        // Attribuer chaque point du dataset à un centroïde initialement aléatoirement
        var newCenters = calculateCenters(clusterIds)
        // Calculer le nouveau centroïde
        while (centroids.sameElements(newCenters)) {
            // Tant que centroids est différent de newCenters :
            centroids = newCenters
            // Enregistrer newCenters dans centroids
            clusterIds = assignToClusters()
            // Assigner les points de data au nouveau centroïde
            newCenters = calculateCenters(clusterIds)
            // Calculer de nouveaux centroïdes newCenters
        }
        clusterIds
        // Retourner ClusterIds
    }
    
    def getCentroids(): Array[Array[Double]] ={
        // Assesseur en lecture concernant le tableau centroids
        return centroids
        // Retourner centroids
    }

    def predict(point: Array[Double]): Int = {
        // Cette méthode renvoie l'indice du cluster 
        // auquel un point donné en argument appartient
        var minDistance = Double.MaxValue
        // Initialiser minDistance à la valeur maximumale
        var clusterIndex = -1
        // Initialiser clusterIndex à -1
        for (i <- centroids.indices) {
            // Pour tout i de centroids.indices :
            val distance = euclideanDistance(point, centroids(i))
            // Calculer la distance euclidienne entre le point et les centroïdes
            if (distance < minDistance) {
                // Si la distance est inférience à minDistance
                minDistance = distance
                // minDistance prend la valeur de distance
                clusterIndex = i
                // Ajouter la valeur de i dans clusterIndex
            }
        }
        clusterIndex
        // Retourner clusterIndex
    }
}

val kMeans = new KMeans("iris.data", 3, 1000)
// Créer une instance de la classe KMeans
val clusters = kMeans.run()
// Exécuter la méthode run implémentant l'algorithme K-Means

println(s"Final clusters: ${clusters.mkString(", ")}")
// Afficher les indices des clusters après exécution de K-Means

Final clusters: 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 0, 2, 1, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0


import scala.io.Source

import scala.util.Random



defined class KMeans
kMeans: KMeans = ammonite.$sess.cmd0$Helper$KMeans@59500943
clusters: Array[Int] = Array(
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
...

In [2]:
val cent = kMeans.getCentroids()
// Renvoie les valeurs des centroïdes

cent: Array[Array[Double]] = Array(
  Array(6.3, 2.5, 4.9, 1.5),
  Array(5.0, 3.5, 1.6, 0.6),
  Array(6.6, 3.0, 4.4, 1.4)
)

In [3]:
val indiceCluster = kMeans.predict(Array[Double](5, 3, 1, 0.5)) 

indiceCluster: Int = 1

### Importation de la bibliothèque graphique plotly pour Almond

In [4]:
import $ivy.`org.plotly-scala::plotly-almond:0.8.2`
// Téléchargement la version de 0.8.2 de plotly-scala pour almond 

import $ivy.$                                      
// Téléchargement la version de 0.8.2 de plotly-scala pour almond 

In [5]:
import plotly._
// Importer tous les sous-modules de Plotly
import plotly.element._
// Importer les définitions pour différents éléments graphiques 
// comme les couleurs, les bordures, etc
import plotly.layout._
// Importer la définition de la mise en page 
// des graphiques : les titres, les légendes, les axes, etc
import plotly.Almond._
// Importer une extension de plotly pour almond sous jupyter notebook

import plotly._
// Importer tous les sous-modules de Plotly

import plotly.element._
// Importer les définitions pour différents éléments graphiques 
// comme les couleurs, les bordures, etc

import plotly.layout._
// Importer la définition de la mise en page 
// des graphiques : les titres, les légendes, les axes, etc

import plotly.Almond._
// Importer une extension de plotly pour almond sous jupyter notebook

### Visualisation de la quantité des éléments de chaque classe
 On peut remarquer que dans le fichier iris.data le nombre de données des iris setosa, celui des iris virginica, et celui des iris versicolor est le même. Ce dernier a pour valeur 50.

In [17]:
val data = scala.io.Source.fromFile("iris.data").getLines.map(_.split(",")).toSeq
// Tokenisation des lignes du fichier iris.data : 

data: Seq[Array[String]] = List(
  Array("5.1", "3.5", "1.4", "0.2", "Iris-setosa"),
  Array("4.9", "3.0", "1.4", "0.2", "Iris-setosa"),
  Array("4.7", "3.2", "1.3", "0.2", "Iris-setosa"),
  Array("4.6", "3.1", "1.5", "0.2", "Iris-setosa"),
  Array("5.0", "3.6", "1.4", "0.2", "Iris-setosa"),
  Array("5.4", "3.9", "1.7", "0.4", "Iris-setosa"),
  Array("4.6", "3.4", "1.4", "0.3", "Iris-setosa"),
  Array("5.0", "3.4", "1.5", "0.2", "Iris-setosa"),
  Array("4.4", "2.9", "1.4", "0.2", "Iris-setosa"),
  Array("4.9", "3.1", "1.5", "0.1", "Iris-setosa"),
  Array("5.4", "3.7", "1.5", "0.2", "Iris-setosa"),
  Array("4.8", "3.4", "1.6", "0.2", "Iris-setosa"),
  Array("4.8", "3.0", "1.4", "0.1", "Iris-setosa"),
  Array("4.3", "3.0", "1.1", "0.1", "Iris-setosa"),
  Array("5.8", "4.0", "1.2", "0.2", "Iris-setosa"),
  Array("5.7", "4.4", "1.5", "0.4", "Iris-setosa"),
  Array("5.4", "3.9", "1.3", "0.4", "Iris-setosa"),
  Array("5.1", "3.5", "1.4", "0.3", "Iris-setosa"),
  Array("5.7", "3.8", "1.7", "0

In [18]:
val irisSetosaCount = data.filter(_(4) == "Iris-setosa").size
// Compter le nombre de lignes contenant la chaîne de caractères : "Iris-setosa"
val irisVersicolorCount = data.filter(_(4) == "Iris-versicolor").size
// Compter le nombre de lignes contenant la chaîne de caractères : "Iris-versicolor"
val irisVirginicaCount = data.filter(_(4) == "Iris-virginica").size
// Compter le nombre de lignes contenant la chaîne de caractères : "Iris-virginica"

val data_iris = Seq(
  Bar(
      // Créer une séquence de barres
    Seq("Iris-setosa", "Iris-versicolor", "Iris-virginica"),
      // pour Iris-sesota, Iris-versicolor, Iris-virginica
    Seq(irisSetosaCount, irisVersicolorCount, irisVirginicaCount)
      // Montrer le nombre d'échantillons pour chaque type d'iris
  )
)

plot(data_iris)
// Afficher le graphique à barres 

irisSetosaCount: Int = 50
irisVersicolorCount: Int = 50
irisVirginicaCount: Int = 50
data_iris: Seq[Bar] = List(
  Bar(
    x = Strings(seq = List("Iris-setosa", "Iris-versicolor", "Iris-virginica")),
    y = Doubles(seq = List(50.0, 50.0, 50.0)),
    name = None,
    text = None,
    marker = None,
    orientation = None,
    xaxis = None,
    yaxis = None,
    error_y = None,
    showlegend = None,
    hoverinfo = None,
    textposition = None,
    opacity = None,
    width = None,
    base = None,
    hovertemplate = None
  )
)
res17_4: String = "plot-f836afd0-bb9c-435d-b7a5-7d92e3639995"

### La représentation de la longueur et de la largeur des sépales pour chaque classe

Cette représentation graphique affiche un nuage de points pour les longueurs et les largeurs de sépales, où les 3 espèces d'iris sont représentées par une couleur différente. On peut remarquer un écart distinct dans les distributions de mesures entre les setosa et les deux autres espèces : versicolor et virginica. En outre,  ces deux dernières s'amalgament très nettemment.

In [19]:
val irisSetosa_x = data.filter(_(4) == "Iris-setosa").map(x => x(0).toDouble)
// Extraire les valeurs correspondant à la première caractéristique 
// de l'ensemble des lignes dédiées aux iris setosa
val irisSetosa_y = data.filter(_(4) == "Iris-setosa").map(x => x(1).toDouble)
// Extraire les valeurs correspondant à la seconde caractéristique 
// de l'ensemble des lignes dédiées aux iris setosa

val irisVersicolor_x = data.filter(_(4) == "Iris-versicolor").map(x => x(0).toDouble)
// Extraire les valeurs correspondant à la première caractéristique 
// de l'ensemble des lignes dédiées aux iris versicolor
val irisVersicolor_y = data.filter(_(4) == "Iris-versicolor").map(x => x(1).toDouble)
// Extraire les valeurs correspondant à la seconde caractéristique 
// de l'ensemble des lignes dédiées aux iris versicolor

val irisVirginica_x = data.filter(_(4) == "Iris-virginica").map(x => x(0).toDouble)
// Extraire les valeurs correspondant à la première caractéristique 
// de l'ensemble des lignes dédiées aux iris virginica
val irisVirginica_y = data.filter(_(4) == "Iris-virginica").map(x => x(1).toDouble)
// Extraire les valeurs correspondant à la seconde caractéristique 
// de l'ensemble des lignes dédiées aux iris virginica

irisSetosa_x: Seq[Double] = List(
  5.1,
  4.9,
  4.7,
  4.6,
  5.0,
  5.4,
  4.6,
  5.0,
  4.4,
  4.9,
  5.4,
  4.8,
  4.8,
  4.3,
  5.8,
  5.7,
  5.4,
  5.1,
  5.7,
  5.1,
  5.4,
  5.1,
  4.6,
  5.1,
  4.8,
  5.0,
  5.0,
  5.2,
  5.2,
  4.7,
  4.8,
  5.4,
  5.2,
  5.5,
  4.9,
  5.0,
  5.5,
  4.9,
...
irisSetosa_y: Seq[Double] = List(
  3.5,
  3.0,
  3.2,
  3.1,
  3.6,
  3.9,
  3.4,
  3.4,
  2.9,
  3.1,
  3.7,
  3.4,
  3.0,
  3.0,
  4.0,
  4.4,
  3.9,
  3.5,
  3.8,
  3.8,
  3.4,
  3.7,
  3.6,
  3.3,
  3.4,
  3.0,
  3.4,
  3.5,
  3.4,
  3.2,
  3.1,
  3.4,
  4.1,
  4.2,
  3.1,
  3.2,
  3.5,
  3.1,
...
irisVersicolor_x: Seq[Double] = List(
  7.0,
  6.4,
  6.9,
  5.5,
  6.5,
  5.7,
  6.3,
  4.9,
  6.6,
  5.2,
  5.0,
  5.9,
  6.0,
  6.1,
  5.6,
  6.7,
  5.6,
  5.8,
  6.2,
  5.6,
  5.9,
  6.1,
  6.3,
  6.1,
  6.4,
  6.6,
  6.8,
  6.7,
  6.0,
  5.7,
  5.5,
  5.5,
  5.8,
  6.0,
  5.4,
  6.0,
  6.7,
  6.3,
...
irisVersicolor_y: Seq[Double] = List(
  3.2,
  3.2,
  3.1,
  2.3,
  2.8,
  2.8,
  3.

In [20]:
val trace1 = Scatter(
    // Créer une instance de la classe Scatter dans le but d'afficher 
    // deux tableaux sous la forme d'un graphique
    irisSetosa_x,
    // Coordonnées x du graphe : un tableau intégrant 
    // la première caractéristique de l'ensemble des iris setosa
    irisSetosa_y,
    // Coordonées y du graphe : un tableau intégrant 
    // la seconde caractéristique de l'ensemble des iris setosa
    mode = ScatterMode(ScatterMode.Markers),
    // Utiliser des marqueurs
    name = "Iris Setosa",
    // Le nom du graphe
    marker = Marker(
        // Couleur et taille des marqueurs et de la ligne 
        // de sélection de chaque marqueur
        color = Color.RGBA(255, 0, 0, 0.8),
        size = 10,
        line = Line(
            color = Color.RGBA(255, 0, 0, 1),
            width = 1
        )
    )
)

val trace2 = Scatter(
    // Créer une instance de la classe Scatter dans le but 
    // d'afficher deux tableaux sous la forme d'un graphique
    irisVersicolor_x,
    // Coordonnées x du graphe : un tableau intégrant 
    // la première caractéristique de l'ensemble des iris versicolor
    irisVersicolor_y,
    // Coordonnées y du graphe : un tableau intégrant 
    // la seconde caractéristique de l'ensemble des iris versicolor
    mode = ScatterMode(ScatterMode.Markers),
    // Utiliser des marqueurs
    name = "Iris Versicolor",
    // Le nom du graphe
    marker = Marker(
        // Couleur et taille des marqueurs et de la ligne 
        // de sélection de chaque marqueur
        color = Color.RGBA(0, 255, 0, 0.8),
        size = 10,
        line = Line(
            color = Color.RGBA(0, 255, 0, 1),
            width = 1
        )
    )
)

val trace3 = Scatter(
    // Créer une instance de la classe Scatter dans le but 
    // d'afficher deux tableaux sous la forme d'un graphique
    irisVirginica_x,
    // Coordonnées x du graphe : un tableau intégrant 
    // la première caractéristique de l'ensemble des iris virginica
    irisVirginica_y,
    // Coordonnées y du graphe : un tableau intégrant 
    // la seconde caractéristique de l'ensemble des iris virginica
    mode = ScatterMode(ScatterMode.Markers),
    // Utiliser des marqueurs
    name = "Iris Virginica",
    // Le nom du graphe
    marker = Marker(
        color = Color.RGBA(0, 0, 255, 0.8),
        // Couleur et taille des marqueurs et de la ligne 
        // de sélection de chaque marqueur
        size = 10,
        line = Line(
            color = Color.RGBA(0, 0, 255, 1),
            width = 1
        )
    )
)

val layout = Layout(
    // Créer une instance Layout dédiée aux propriétés du graphe
    title = "Iris Data",
    // Le titre du graphe
    xaxis = Axis(
        title = "Sepal Length (cm)"
        // Le titre de l'axe des x
    ),
    yaxis = Axis(
        title = "Sepal width (cm)"
        // Le titre de l'axe des y
    )
)

plot(Seq(trace1, trace2, trace3), layout)
// Afficher le graphe et sa mise en page


trace1: Scatter = Scatter(
  x = Some(
    value = Doubles(
      seq = List(
        5.1,
        4.9,
        4.7,
        4.6,
        5.0,
        5.4,
        4.6,
        5.0,
        4.4,
        4.9,
        5.4,
        4.8,
        4.8,
        4.3,
        5.8,
        5.7,
        5.4,
        5.1,
        5.7,
        5.1,
        5.4,
        5.1,
        4.6,
        5.1,
        4.8,
        5.0,
        5.0,
        5.2,
        5.2,
        4.7,
        4.8,
        5.4,
        5.2,
        5.5,
        4.9,
...
trace2: Scatter = Scatter(
  x = Some(
    value = Doubles(
      seq = List(
        7.0,
        6.4,
        6.9,
        5.5,
        6.5,
        5.7,
        6.3,
        4.9,
        6.6,
        5.2,
        5.0,
        5.9,
        6.0,
        6.1,
        5.6,
        6.7,
        5.6,
        5.8,
        6.2,
        5.6,
        5.9,
        6.1,
        6.3,
        6.1,
        6.4,
        6.6,
        6.8,
        6.7,
        6.0,
        

# La représentation de la longueur et de la largeur des pétales pour chaque classe

Cette représentation graphique affiche un nuage de points pour les longueurs et les largeurs de pétales. La dispersion des classes est nettement plus marquée dans cette représentation. Cependant le regroupement des iris versicolor et celui des iris virginica sont très rapprochés. Dans ces dernières, quelques données ont tendances à se mélanger.

In [22]:
val irisSetosa_x = data.filter(_(4) == "Iris-setosa").map(x => x(2).toDouble)
// Extraire les valeurs correspondant à la troisième caractéristique 
// de l'ensemble des lignes dédiées aux iris setosa
val irisSetosa_y = data.filter(_(4) == "Iris-setosa").map(x => x(3).toDouble)
// Extraire les valeurs correspondant à la quatrième caractéristique 
// de l'ensemble des lignes dédiées aux iris setosa

val irisVersicolor_x = data.filter(_(4) == "Iris-versicolor").map(x => x(2).toDouble)
// Extraire les valeurs correspondant à la troisième caractéristique 
// de l'ensemble des lignes dédiées aux iris versicolor
val irisVersicolor_y = data.filter(_(4) == "Iris-versicolor").map(x => x(3).toDouble)
// Extraire les valeurs correspondant à la quatrième caractéristique 
// de l'ensemble des lignes dédiées aux iris versicolor

val irisVirginica_x = data.filter(_(4) == "Iris-virginica").map(x => x(2).toDouble)
// Extraire les valeurs correspondant à la troisième caractéristique 
// de l'ensemble des lignes dédiées aux iris virginica
val irisVirginica_y = data.filter(_(4) == "Iris-virginica").map(x => x(3).toDouble)
// Extraire les valeurs correspondant à la quatrième caractéristique 
// de l'ensemble des lignes dédiées aux iris virginica

irisSetosa_x: Seq[Double] = List(
  1.4,
  1.4,
  1.3,
  1.5,
  1.4,
  1.7,
  1.4,
  1.5,
  1.4,
  1.5,
  1.5,
  1.6,
  1.4,
  1.1,
  1.2,
  1.5,
  1.3,
  1.4,
  1.7,
  1.5,
  1.7,
  1.5,
  1.0,
  1.7,
  1.9,
  1.6,
  1.6,
  1.5,
  1.4,
  1.6,
  1.6,
  1.5,
  1.5,
  1.4,
  1.5,
  1.2,
  1.3,
  1.5,
...
irisSetosa_y: Seq[Double] = List(
  0.2,
  0.2,
  0.2,
  0.2,
  0.2,
  0.4,
  0.3,
  0.2,
  0.2,
  0.1,
  0.2,
  0.2,
  0.1,
  0.1,
  0.2,
  0.4,
  0.4,
  0.3,
  0.3,
  0.3,
  0.2,
  0.4,
  0.2,
  0.5,
  0.2,
  0.2,
  0.4,
  0.2,
  0.2,
  0.2,
  0.2,
  0.4,
  0.1,
  0.2,
  0.1,
  0.2,
  0.2,
  0.1,
...
irisVersicolor_x: Seq[Double] = List(
  4.7,
  4.5,
  4.9,
  4.0,
  4.6,
  4.5,
  4.7,
  3.3,
  4.6,
  3.9,
  3.5,
  4.2,
  4.0,
  4.7,
  3.6,
  4.4,
  4.5,
  4.1,
  4.5,
  3.9,
  4.8,
  4.0,
  4.9,
  4.7,
  4.3,
  4.4,
  4.8,
  5.0,
  4.5,
  3.5,
  3.8,
  3.7,
  3.9,
  5.1,
  4.5,
  4.5,
  4.7,
  4.4,
...
irisVersicolor_y: Seq[Double] = List(
  1.4,
  1.5,
  1.5,
  1.3,
  1.5,
  1.3,
  1.

### La représentation de la longueur et de la largeur des pétales pour chaque classe

Cette représentation graphique affiche un nuage de points pour les longueurs et largeurs de pétales. La dispersion des classes est nettement plus marquée dans cette représentation. Cependant le regroupement des iris versicolor et celui des iris virginica sont très rapprochés. Dans ces dernières, quelques données ont tendances à se mélanger.

In [23]:
val trace1 = Scatter(
    // Créer une instance de la classe Scatter dans le but 
    // d'afficher deux tableaux sous la forme d'un graphique
    irisSetosa_x,
    // Coordonnées x du graphe : un tableau intégrant la troisième caractéristique 
    // de l'ensemble des iris virginica
    irisSetosa_y,
    // Coordonnées y du graphe : un tableau intégrant la quatrième caractéristique 
    // de l'ensemble des iris virginica
    mode = ScatterMode(ScatterMode.Markers),
    name = "Iris Setosa",
    // Le nom du graphe
    marker = Marker(
        // Utiliser des marqueurs
        color = Color.RGBA(255, 0, 0, 0.8),
        // Couleur et taille des marqueurs et de la ligne de sélection de chaque marqueur
        size = 10,
        line = Line(
            color = Color.RGBA(255, 0, 0, 1),
            width = 1
        )
    )
)

val trace2 = Scatter(
    // Créer une instance de la classe Scatter dans le but 
    // d'afficher deux tableaux sous la forme d'un graphique
    irisVersicolor_x,
    // Coordonnées x du graphe : un tableau intégrant la troisième caractéristique 
    // de l'ensemble des iris versicolor
    irisVersicolor_y,
    // Coordonnées y du graphe : un tableau intégrant la quatrième caractéristique 
    // de l'ensemble des iris versicolor
    mode = ScatterMode(ScatterMode.Markers),
    name = "Iris Versicolor",
    // Le nom du graphe
    marker = Marker(
        // Utiliser des marqueurs
        color = Color.RGBA(0, 255, 0, 0.8),
        // Couleur et taille des marqueurs et de la ligne de sélection de chaque marqueur
        size = 10,
        line = Line(
            color = Color.RGBA(0, 255, 0, 1),
            width = 1
        )
    )
)

val trace3 = Scatter(
    // Créer une instance de la classe Scatter dans le but d'afficher 
    // deux tableaux sous la forme d'un graphique
    irisVirginica_x,
    // Coordonnées x du graphe : un tableau intégrant la troisième caractéristique 
    // de l'ensemble des iris virginica
    irisVirginica_y,
    // Coordonnées x du graphe : un tableau intégrant la quatrième caractéristique 
    // de l'ensemble des iris virginica
    mode = ScatterMode(ScatterMode.Markers),
    name = "Iris Virginica",
    // Le nom du graphe
    marker = Marker(
        // Utiliser des marqueurs
        color = Color.RGBA(0, 0, 255, 0.8),
        // Couleur et taille des marqueurs et de la ligne de sélection de chaque marqueur
        size = 10,
        line = Line(
            color = Color.RGBA(0, 0, 255, 1),
            width = 1
        )
    )
)

val layout = Layout(
    // Créer une instance Layout dédiée aux propriétés du graphe
    title = "Iris Data",
    // Le titre du graphe
    xaxis = Axis(
        title = "Petal Length (cm)"
        // Le titre l'axe des x
    ),
    yaxis = Axis(
        title = "Petal width (cm)"
        // Le titre l'axe des y
    )
)

plot(Seq(trace1, trace2, trace3), layout)
// Afficher le graphe et sa mise en page


trace1: Scatter = Scatter(
  x = Some(
    value = Doubles(
      seq = List(
        1.4,
        1.4,
        1.3,
        1.5,
        1.4,
        1.7,
        1.4,
        1.5,
        1.4,
        1.5,
        1.5,
        1.6,
        1.4,
        1.1,
        1.2,
        1.5,
        1.3,
        1.4,
        1.7,
        1.5,
        1.7,
        1.5,
        1.0,
        1.7,
        1.9,
        1.6,
        1.6,
        1.5,
        1.4,
        1.6,
        1.6,
        1.5,
        1.5,
        1.4,
        1.5,
...
trace2: Scatter = Scatter(
  x = Some(
    value = Doubles(
      seq = List(
        4.7,
        4.5,
        4.9,
        4.0,
        4.6,
        4.5,
        4.7,
        3.3,
        4.6,
        3.9,
        3.5,
        4.2,
        4.0,
        4.7,
        3.6,
        4.4,
        4.5,
        4.1,
        4.5,
        3.9,
        4.8,
        4.0,
        4.9,
        4.7,
        4.3,
        4.4,
        4.8,
        5.0,
        4.5,
        